In [ ]:
import os

os.environ["HF_TOKEN"] = ""

In [ ]:
!pip install -q transformers accelerate bitsandbytes datasets torch safetensors peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/685-Project"

Mounted at /content/drive


In [ ]:
# ============================================================================
# TOKEN BUDGET
# ============================================================================

class TokenBudget:
    """
    Simple token budget tracker.
    """
    def __init__(self, max_tokens: int):
        self.max_tokens = max_tokens
        self.used_tokens = 0

    def spend(self, n_tokens: int, label: str = ""):
        self.used_tokens += n_tokens
        tag = f" ({label})" if label else ""
        print(
            f"[TokenBudget] +{n_tokens}{tag}. "
            f"Total: {self.used_tokens}/{self.max_tokens}"
        )

    @property
    def exhausted(self) -> bool:
        return self.used_tokens >= self.max_tokens

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
import json
from typing import List, Dict, Tuple
from peft import PeftModel

# ============================================================================
# MODEL LOADING WITH 4-BIT QUANTIZATION
# ============================================================================

def load_models():

    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

    print("Loading Llama 3.1 8B (Step Generator)...")
    decomp_tokenizer = AutoTokenizer.from_pretrained("unsloth/meta-Llama-3.1-8B-Instruct")
    decomp_base_model = AutoModelForCausalLM.from_pretrained(
        "unsloth/meta-Llama-3.1-8B-Instruct",
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True
    )

    decomp_adapter_path = "/content/drive/MyDrive/685-Project/SFT-Decomposer/checkpoint-2138"

    decomp_model = PeftModel.from_pretrained(decomp_base_model, decomp_adapter_path)


    print("Loading Llama 3.1 8B (Step Evaluator)...")
    eval_tokenizer = AutoTokenizer.from_pretrained("unsloth/meta-Llama-3.1-8B-Instruct")
    eval_base_model = AutoModelForCausalLM.from_pretrained(
        "unsloth/meta-Llama-3.1-8B-Instruct",
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True
    )

    eval_adapter_path = "/content/drive/MyDrive/685-Project/SFT-Evaluator/sft-saved-lora"

    eval_model = PeftModel.from_pretrained(eval_base_model, eval_adapter_path)

    # Some chat models don't have pad_token set; make it eos to avoid warnings
    if decomp_tokenizer.pad_token_id is None:
        decomp_tokenizer.pad_token_id = decomp_tokenizer.eos_token_id
    if eval_tokenizer.pad_token_id is None:
        eval_tokenizer.pad_token_id = eval_tokenizer.eos_token_id

    return decomp_model, decomp_tokenizer, eval_model, eval_tokenizer

# Load models
decomp_model, decomp_tokenizer, eval_model, eval_tokenizer = load_models()

Loading Llama 3.1 8B (Step Generator)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Loading Llama 3.1 8B (Step Evaluator)...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def generate_next_step(
    problem: str,
    steps_so_far: List[str],
    decomp_model,
    decomp_tokenizer,
    hint: str = None,
    rejected_step: str = None,
    max_new_tokens: int = 50,
    budget: TokenBudget | None = None,
) -> str:
    # -------- 1) Clean and normalize steps_so_far into plain step texts --------
    clean_steps = []
    for s in steps_so_far:
        if not s.strip():
            continue

        # Never include FINAL_ANSWER in the history
        if s.strip().upper().startswith("FINAL_ANSWER"):
            continue

        # Extract content after "STEP:" if present
        step_text = None
        for line in s.splitlines():
            line = line.strip()
            if line.upper().startswith("STEP:"):
                step_text = line.split("STEP:", 1)[1].strip()
                break
        if step_text is None:
            step_text = s.strip()

        clean_steps.append(step_text)

    # -------- 2) Build history string EXACTLY like in the training CSV --------
    if clean_steps:
        context = ""
        for i, h in enumerate(clean_steps, start=1):
            context += f"STEP {i}: {h}\n"
        context = context.rstrip()
    else:
        context = ""

    # -------- 3) Build the input text in the same pattern as training + retry info --------
    INPUT_TEXT = (
        f"Problem: {problem}\n\n"
        f"Steps completed so far:\n{context}"
    )

    # If we are retrying a specific (rejected) step, show it explicitly
    if rejected_step:
        INPUT_TEXT += f"\n\nLast attempted step (INCORRECT):\n{rejected_step}"

    if hint:
        INPUT_TEXT += (
            f"\n\nHint from evaluator: {hint}\n"
            "Read the hint carefully and rewrite ONLY that incorrect step."
            "Do not change the previous correct steps."
        )

    SYSTEM_PROMPT = """
You are a careful stepwise math solver. Read and understand the problem carefully.
When a last attempted step is marked INCORRECT with a hint, you must generate a corrected version of that step.
""".strip()

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": INPUT_TEXT},
    ]

    input_ids = decomp_tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(decomp_model.device)

    with torch.no_grad():
        outputs = decomp_model.generate(
            input_ids,
            max_new_tokens=max_new_tokens,
            temperature=0.1,
            do_sample=False,
            pad_token_id=decomp_tokenizer.pad_token_id,
        )

    # count tokens
    if budget is not None:
        input_len = input_ids.shape[1]
        total_len = outputs[0].shape[0]
        gen_len = total_len - input_len
        budget.spend(gen_len, label="Decomposer")

    response = decomp_tokenizer.decode(
        outputs[0][input_ids.shape[1]:],
        skip_special_tokens=True,
    ).strip()

    return response

In [ ]:
def evaluate_step(
    problem: str,
    steps_so_far: List[str],
    current_step: str,
    eval_model,
    eval_tokenizer,
    budget: TokenBudget | None = None,
) -> Tuple[Dict[str, float], str]:

    SYSTEM_PROMPT = """
You are a strict math-step evaluator. You ONLY judge the CURRENT step,
given the problem and the previous steps.

You must output a single integer score from 0 to 3:

3 = Good step:
    - Mathematically correct.
    - Logically follows from previous steps.
    - Clearly helps move toward the final answer.

2 = Almost good:
    - Mostly correct and helpful.
    - Might have a small issue (slight imprecision or missing minor detail),
      but the step is still usable.

1 = Weak / partially correct:
    - Contains some relevant or correct ideas,
      but has an important mistake or is missing a key part.

0 = Bad step:
    - Wrong math, wrong logic, irrelevant, or seriously misleading.
    - Should NOT be used as part of the final solution.
    - Same content as the previous step

You MUST recompute any numeric calculation in the current step yourself.
If the arithmetic is wrong, the score should usually be 0 or 1.

Output EXACTLY TWO lines in this format:

Score: <0 or 1 or 2 or 3>
Feedback: <one short sentence about why the step is good or bad>
""".strip()

    # Build context
    context = f"Problem: {problem}\n\n"
    if steps_so_far:
        context += "Previous steps:\n"
        for i, step in enumerate(steps_so_far, 1):
            # Pretty-print previous steps
            s = step.strip()
            if s.upper().startswith("STEP:"):
                s = s.split("STEP:", 1)[1].strip()
            context += f"Step {i}: {s}\n"
        context += "\n"

    context += f"Current step to evaluate:\n{current_step}\n\n"

    instance_instruction = (
        context
        + "Now give your evaluation.\n"
        + "Remember: only these two lines, nothing more and nothing less:\n"
        + "Score: <0 or 1 or 2 or 3>\n"
        + "Feedback: <one sentence>\n"
    )

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": instance_instruction},
    ]

    input_ids = eval_tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to(eval_model.device)

    with torch.no_grad():
        outputs = eval_model.generate(
            input_ids,
            max_new_tokens=64,
            temperature=0.0,
            do_sample=False,
            pad_token_id=eval_tokenizer.pad_token_id,
        )

    # count tokens
    if budget is not None:
        input_len = input_ids.shape[1]
        total_len = outputs[0].shape[0]
        gen_len = total_len - input_len
        budget.spend(gen_len, label="Evaluator")

    response = eval_tokenizer.decode(
        outputs[0][input_ids.shape[1]:],
        skip_special_tokens=True,
    ).strip()

    # ---- Parse score + feedback ----
    raw_score = None
    feedback = ""

    for line in response.splitlines():
        if ":" not in line:
            continue
        key, value = line.split(":", 1)
        key = key.strip().lower()
        value = value.strip()

        if key == "score":
            # grab the first integer in the line
            m = re.search(r"\d+", value)
            if m:
                raw_score = int(m.group(0))
        elif key == "feedback":
            feedback = value

    # Fallbacks
    if raw_score is None:
        raw_score = 1  # conservative mediocre default
    if not feedback:
        feedback = "Step quality is unclear; treating as mediocre."

    scores = {
        "Raw_Score": float(raw_score),
    }

    return scores, feedback

In [ ]:
# ============================================================================
# COLLABORATIVE SOLVER
# ============================================================================

def solve_problem_collaboratively(
    problem: str,
    decomp_model,
    decomp_tokenizer,
    eval_model,
    eval_tokenizer,
    max_steps: int = 10,
    score_threshold: float = 2.0,
    max_retries: int = 2,
    max_token_budget=1400
) -> Dict:
    steps: List[str] = []
    all_evaluations: List[Dict] = []
    final_answer = None

    print(f"\n{'='*80}")
    print(f"PROBLEM: {problem}")
    print(f"{'='*80}\n")

    budget = TokenBudget(max_tokens=max_token_budget)

    for step_idx in range(1, max_steps + 1):

        if budget.exhausted:
            print("[TokenBudget] Exhausted before starting this step. Stopping.")
            break

        rejected_step = None  # for this logical step

        for retry in range(max_retries + 1):
            if budget.exhausted:
                print("[TokenBudget] Exhausted before generation. Stopping.")
                break

            if retry == 0:
                print(f"\n--- Generating Step {step_idx} ---")
                hint = None
            else:
                print(f"\n--- Re-Generating Step {step_idx} (retry {retry}) ---")
                # use last evaluation's feedback as hint
                hint = all_evaluations[-1].get("feedback", None) if all_evaluations else None

            # Generate step from decomposer
            current_step = generate_next_step(
                problem,
                steps,            # only accepted steps
                decomp_model,
                decomp_tokenizer,
                hint=hint,
                rejected_step=rejected_step,
                budget=budget
            )

            print(current_step)
            print()

            # If the model outputs FINAL_ANSWER directly, accept and stop
            if current_step.strip().upper().startswith("FINAL_ANSWER"):
                match = re.search(r"FINAL_ANSWER\s*:\s*(.+)", current_step, re.IGNORECASE)
                if match:
                    final_answer = match.group(1).strip()
                steps.append(current_step)
                print(f"✓ Final answer reached: {final_answer}")
                # Break out of both loops
                return {
                    "problem": problem,
                    "final_steps": steps,
                    "all_evaluations": all_evaluations,
                    "num_steps": len(steps),
                    "final_answer": final_answer,
                }

            # --- Evaluate current step ---
            print("--- Evaluating Step ---")
            scores, feedback = evaluate_step(
                problem,
                steps,          # only accepted steps so far
                current_step,   # step we're grading now
                eval_model,
                eval_tokenizer,
                budget=budget
            )

            raw_score = scores.get("Raw_Score", None)
            if raw_score is None:
                raw_score = 1.0  # conservative default if evaluator glitches

            # Clamp to [0,3] to be safe
            raw_score = max(0.0, min(float(raw_score), 3.0))

            print(f"Raw score: {raw_score:.1f}/3")
            print(f"Feedback: {feedback}")

            evaluation = {
                "step_number": step_idx,
                "step": current_step,
                "scores": scores,
                "feedback": feedback,
                "retry_attempt": retry,
            }
            all_evaluations.append(evaluation)

            # --- Accept / Retry logic based on Raw_Score ---
            if raw_score >= score_threshold:
                # Accept step
                steps.append(current_step)
                print("✓ Step accepted!")

                # Check for FINAL_ANSWER in an accepted step
                match = re.search(r"FINAL_ANSWER\s*:\s*(.+)", current_step, re.IGNORECASE)
                if match:
                    final_answer = match.group(1).strip()
                    print(f"\n✓ Final answer reached: {final_answer}")
                    return {
                        "problem": problem,
                        "final_steps": steps,
                        "all_evaluations": all_evaluations,
                        "num_steps": len(steps),
                        "final_answer": final_answer,
                    }

                # Break out of retry loop, move to next logical step
                break

            else:
                # raw_score < threshold → bad/weak step
                if retry >= max_retries:
                    print("⚠ Max retries reached, accepting step and continuing...")
                    steps.append(current_step)
                    # Move on to next logical step
                    break
                else:
                    print(f"✗ Step needs improvement (retry {retry + 1}/{max_retries})")
                    rejected_step = current_step   # tell decomposer what exactly to fix
                    # retry loop continues

            if budget.exhausted:
                print("[TokenBudget] Exhausted after this step loop. Stopping.")
                break

        # end inner retry loop

    # If we exit the outer loop without hitting FINAL_ANSWER
    print("\n⚠ Reached max_steps without a FINAL_ANSWER.")
    return {
        "problem": problem,
        "final_steps": steps,
        "all_evaluations": all_evaluations,
        "num_steps": len(steps),
        "final_answer": final_answer,
    }

In [ ]:
# ============================================================================
# MAIN EXECUTION
# ============================================================================

# Test on a single problem
test_problem = (
    "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning "
    "and bakes muffins for her friends every day with four. She sells the remainder "
    "at the farmers' market daily for $2 per fresh duck egg. How much in dollars "
    "does she make every day at the farmers' market?"
)

result = solve_problem_collaboratively(
    test_problem,
    decomp_model,
    decomp_tokenizer,
    eval_model,
    eval_tokenizer
)

print("\n\n" + "="*80)
print("FINAL SOLUTION SUMMARY")
print("="*80)

for i, step in enumerate(result['final_steps'], 1):
    print(f"\nStep {i}: {step}")

print(f"\nParsed Final Answer: {result['final_answer']}")


PROBLEM: Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?


--- Generating Step 1 ---
[TokenBudget] +36 (Decomposer). Total: 36/1600
STEP: First find the number of eggs that Janet uses for muffins: 3 eggs/day + 4 eggs/day = <<3+4=7>>7 eggs/day

--- Evaluating Step ---
[TokenBudget] +26 (Evaluator). Total: 62/1600
Raw score: 0.0/3
Feedback: You forgot to subtract the eggs used for muffins from the total number of eggs laid.
✗ Step needs improvement (retry 1/2)

--- Re-Generating Step 1 (retry 1) ---
[TokenBudget] +39 (Decomposer). Total: 101/1600
STEP: Subtract the number of eggs used for muffins from the total number of eggs: 16 eggs/day - 7 eggs/day = <<16-7=9>>9 eggs/day

--- Evaluating Step ---
[TokenBudget] +43 (Evaluator). Total: 144/1600
Raw score: 2.

In [ ]:
import re
from datasets import load_dataset

# Helper: extract gold final answer from GSM8K "answer" field
def extract_gsm8k_final_answer(answer_text: str) -> str | None:
    m = re.search(r"####\s*(.+)", answer_text)
    if not m:
        return None
    return m.group(1).strip()

# Helper: normalize numeric answers for comparison
def normalize_numeric_answer(ans: str | None) -> str | None:
    if ans is None:
        return None

    text = ans.replace(",", "").replace("$", "").strip()
    m = re.search(r"-?\d+(\.\d+)?", text)
    if not m:
        return text if text else None
    return m.group(0)


# ============================================================================
# GSM8K TESTING
# ============================================================================

def test_on_gsm8k(
    decomp_model,
    decomp_tokenizer,
    eval_model,
    eval_tokenizer,
    num_samples: int = 5
):
    print("Loading GSM8K dataset...")
    # Use whatever slice you like; this is your current choice
    dataset = load_dataset("gsm8k", "main", split=f"test[:{num_samples}]")

    results = []
    num_correct = 0
    num_total = 0

    for idx in range(min(num_samples, len(dataset))):
        problem = dataset[idx]["question"]
        ground_truth = dataset[idx]["answer"]

        result = solve_problem_collaboratively(
            problem,
            decomp_model,
            decomp_tokenizer,
            eval_model,
            eval_tokenizer
        )

        result["ground_truth"] = ground_truth
        results.append(result)

        model_final = result.get("final_answer", None)
        gold_final = extract_gsm8k_final_answer(ground_truth)

        norm_model = normalize_numeric_answer(model_final)
        norm_gold = normalize_numeric_answer(gold_final)

        is_correct = (norm_model is not None
                      and norm_gold is not None
                      and norm_model == norm_gold)

        num_total += 1
        if is_correct:
            num_correct += 1

        print(f"\n{'='*80}")
        print(f"Completed problem {idx+1}/{num_samples}")
        print(f"Model final answer: {model_final}  (normalized: {norm_model})")
        print(f"Ground truth: {gold_final}        (normalized: {norm_gold})")
        print(f"Correct? {is_correct}")
        print(f"{'='*80}\n")

        print("\n\n" + "="*80)
        print("FINAL SOLUTION SUMMARY")
        print("="*80)
        for j, step in enumerate(result["final_steps"], 1):
            print(f"\nStep {j}: {step}")

    # ---- Compute and print accuracy ----
    if num_total > 0:
        accuracy = num_correct / num_total
    else:
        accuracy = 0.0

    print("\n" + "="*80)
    print(f"Evaluated {num_total} GSM8K problems.")
    print(f"Correct: {num_correct}")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("="*80 + "\n")

In [ ]:
test_on_gsm8k(
    decomp_model,
    decomp_tokenizer,
    eval_model,
    eval_tokenizer,
    num_samples=50
)

Loading GSM8K dataset...


README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



PROBLEM: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?


--- Generating Step 1 ---
[TokenBudget] +39 (Decomposer). Total: 39/1600
STEP: First find the number of eggs that Janet eats and bakes into muffins: 3 eggs/day + 4 eggs/day = <<3+4=7>>7 eggs/day

--- Evaluating Step ---
[TokenBudget] +29 (Evaluator). Total: 68/1600
Raw score: 1.0/3
Feedback: The math is correct, but the step is weak because it doesn't help move toward the final answer.
✗ Step needs improvement (retry 1/2)

--- Re-Generating Step 1 (retry 1) ---
[TokenBudget] +41 (Decomposer). Total: 109/1600
STEP: Then subtract that number from the number of eggs laid to find the number of eggs sold: 16 eggs/day - 7 eggs/day = <<16-7=9>>9 eggs/day

--- Evaluating Step ---
[TokenBudget] +27 (Evaluat

##Only Decomposer

In [ ]:
def solve_problem_decomposer_only(
    problem: str,
    decomp_model,
    decomp_tokenizer,
    max_steps: int = 10,
) -> Dict:

    steps: List[str] = []
    final_answer = None

    print(f"\n{'='*80}")
    print(f"PROBLEM: {problem}")
    print(f"{'='*80}\n")

    for step_idx in range(1, max_steps + 1):
        print(f"\n--- Generating Step {step_idx} ---")

        # No hint (since no evaluator); just pass history
        current_step = generate_next_step(
            problem,
            steps,
            decomp_model,
            decomp_tokenizer,
            hint=None
        )

        print(current_step)
        print()

        # If the model outputs FINAL_ANSWER directly, accept and stop
        if current_step.strip().upper().startswith("FINAL_ANSWER"):
            match = re.search(r"FINAL_ANSWER\s*:\s*(.+)", current_step, re.IGNORECASE)
            if match:
                final_answer = match.group(1).strip()
            steps.append(current_step)
            print(f"✓ Final answer reached: {final_answer}")
            break

        # Otherwise, treat it as a normal step and continue
        steps.append(current_step)

    # If we exit the loop without seeing FINAL_ANSWER
    if final_answer is None:
        print("\n⚠ Reached max_steps without a FINAL_ANSWER.")

    return {
        "problem": problem,
        "final_steps": steps,
        "all_evaluations": [],     # no evaluator, so keep this empty for compatibility
        "num_steps": len(steps),
        "final_answer": final_answer,
    }

In [ ]:
def test_on_gsm8k_decomp_only(
    decomp_model,
    decomp_tokenizer,
    num_samples: int = 5
):
    print("Loading GSM8K dataset...")
    # Use whatever slice you like; this is your current choice
    dataset = load_dataset("gsm8k", "main", split=f"test[:{num_samples}]")

    results = []
    num_correct = 0
    num_total = 0

    for idx in range(min(num_samples, len(dataset))):
        problem = dataset[idx]["question"]
        ground_truth = dataset[idx]["answer"]

        result = solve_problem_decomposer_only(
            problem,
            decomp_model,
            decomp_tokenizer,
        )

        result["ground_truth"] = ground_truth
        results.append(result)

        model_final = result.get("final_answer", None)
        gold_final = extract_gsm8k_final_answer(ground_truth)

        norm_model = normalize_numeric_answer(model_final)
        norm_gold = normalize_numeric_answer(gold_final)

        is_correct = (norm_model is not None
                      and norm_gold is not None
                      and norm_model == norm_gold)

        num_total += 1
        if is_correct:
            num_correct += 1

        print(f"\n{'='*80}")
        print(f"Completed problem {idx+1}/{num_samples}")
        print(f"Model final answer: {model_final}  (normalized: {norm_model})")
        print(f"Ground truth: {gold_final}        (normalized: {norm_gold})")
        print(f"Correct? {is_correct}")
        print(f"{'='*80}\n")

        print("\n\n" + "="*80)
        print("FINAL SOLUTION SUMMARY")
        print("="*80)
        for j, step in enumerate(result["final_steps"], 1):
            print(f"\nStep {j}: {step}")

    # ---- After all problems, compute and print accuracy ----
    if num_total > 0:
        accuracy = num_correct / num_total
    else:
        accuracy = 0.0

    print("\n" + "="*80)
    print(f"Evaluated {num_total} GSM8K problems.")
    print(f"Correct: {num_correct}")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("="*80 + "\n")

    # return results, accuracy

In [ ]:
test_on_gsm8k_decomp_only(
    decomp_model,
    decomp_tokenizer,
    num_samples=50
)

Loading GSM8K dataset...

PROBLEM: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?


--- Generating Step 1 ---
STEP: First find the number of eggs Janet uses for breakfast and baking: 3 eggs + 4 eggs = <<3+4=7>>7 eggs


--- Generating Step 2 ---
STEP: Then subtract that number from the number of eggs laid to find the number of eggs sold: 16 eggs - 7 eggs = <<16-7=9>>9 eggs


--- Generating Step 3 ---
STEP: Then multiply the number of eggs sold by the price per egg to find Janet's daily earnings: 9 eggs * $2/egg = $<<9*2=18>>18


--- Generating Step 4 ---
FINAL_ANSWER: 18

✓ Final answer reached: 18

Completed problem 1/50
Model final answer: 18  (normalized: 18)
Ground truth: 18        (normalized: 18)
Correct? True



FINAL SOLUTION SUMMARY

Step 1: STEP: 